# Scoping

In [1]:
import os
from io import BytesIO, StringIO
from pathlib import Path

import boto3
import botocore.exceptions
import pandas as pd
from dotenv import load_dotenv

In [2]:
PROJ_ROOT = Path.cwd().parent

In [3]:
assert load_dotenv(dotenv_path=PROJ_ROOT.parent / '.env')

## About

Project scoping using the full random sample of data for ~10,000 customers from McMaster Bank.

## User Inputs

In [4]:
# R2 data bucket details
bucket_name = 'cc-churn-splits'

# columns to load for project scoping tasks
columns = [
    'clientnum',
    'card_category',
    'total_revolv_bal',
    'total_trans_amt',
    'is_churned',
]

# costs
# # revenue from transactions (bank earns #% of transaction volume)
interchange_rate = 0.02
# # revenue from revolving balance (~20% interest)
apr = 0.18
# # fee revenue from credit card exposure (modeled from card type)
card_fees = {"Blue": 0, "Silver": 50, "Gold": 100, "Platinum": 200}
tenure_years = 3
discount = 0.9
# # percentage of churners who can be convinced to stay (i.e. success rate
# # of saving a churning customer)
success_rate = 0.40
# # cost of intervention to get a single customer to not churn (discounts,
# # call center time, retention offers, etc.)
intervention_cost = 50
# # cost of acquiring a new customer (Customer Acquisition Cost, CAC)
replacement_cost = 200

In [5]:
account_id = os.getenv('ACCOUNT_ID')
access_key_id = os.getenv('ACCESS_KEY_ID')
secret_access_key = os.getenv('SECRET_ACCESS_KEY')

s3_client = boto3.client(
    's3',
    endpoint_url=f'https://{account_id}.r2.cloudflarestorage.com',
    aws_access_key_id=access_key_id,
    aws_secret_access_key=secret_access_key,
    region_name='auto'
)

multiplier = (1 - discount**tenure_years) / (1 - discount)

## Load Data

In [6]:
%%time
dfs = []
response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix='')
for obj in response.get('Contents', []):
    if obj['Key'].endswith('.parquet.gzip'):
        obj = s3_client.get_object(Bucket=bucket_name, Key=obj['Key'])
        df = pd.read_parquet(BytesIO(obj['Body'].read()), columns=columns)
        dfs.append(df)
df = pd.concat(dfs, ignore_index=True).astype({"is_churned": 'bool[pyarrow]'})
df

CPU times: user 121 ms, sys: 13.7 ms, total: 134 ms
Wall time: 474 ms


,clientnum,card_category,total_revolv_bal,total_trans_amt,is_churned
0,721418133,Blue,972.0,4535.0,False
1,719513733,Blue,0.0,4276.0,False
2,721043058,Blue,2149.0,1700.0,False
3,718435158,Blue,0.0,1930.0,False
4,711336033,Blue,1032.0,4080.0,False
...,...,...,...,...,...
10122,713366583,Blue,1685.0,1405.0,False
10123,789684258,Silver,1847.0,7842.0,True
10124,714115533,Blue,1789.0,2591.0,False
10125,767574108,Blue,786.0,4094.0,False


## Calculate Impact of Churn

Calculate customer lifetime value (CLV)

In [7]:
%%time
df = df.assign(
    interchange_rev=lambda df: df["total_trans_amt"] * interchange_rate,
    interest_rev=lambda df: df["total_revolv_bal"] * apr,
    fee_rev=lambda df: (
        df["card_category"].map(card_fees).astype('int16[pyarrow]')
    ),
    annual_rev=lambda df: (
        df["interchange_rev"] + df["interest_rev"] + df["fee_rev"]
    ),
    clv=lambda df: df["annual_rev"] * multiplier,
    intervention_cost=intervention_cost,
)
df

CPU times: user 4.75 ms, sys: 0 ns, total: 4.75 ms
Wall time: 4.35 ms


,clientnum,card_category,total_revolv_bal,total_trans_amt,is_churned,interchange_rev,interest_rev,fee_rev,annual_rev,clv,intervention_cost
0,721418133,Blue,972.0,4535.0,False,90.7,174.96,0,265.66,719.9386,50
1,719513733,Blue,0.0,4276.0,False,85.52,0.0,0,85.52,231.7592,50
2,721043058,Blue,2149.0,1700.0,False,34.0,386.82,0,420.82,1140.4222,50
3,718435158,Blue,0.0,1930.0,False,38.6,0.0,0,38.6,104.606,50
4,711336033,Blue,1032.0,4080.0,False,81.6,185.76,0,267.36,724.5456,50
...,...,...,...,...,...,...,...,...,...,...,...
10122,713366583,Blue,1685.0,1405.0,False,28.1,303.3,0,331.4,898.094,50
10123,789684258,Silver,1847.0,7842.0,True,156.84,332.46,50,539.3,1461.503,50
10124,714115533,Blue,1789.0,2591.0,False,51.82,322.02,0,373.84,1013.1064,50
10125,767574108,Blue,786.0,4094.0,False,81.88,141.48,0,223.36,605.3056,50


The class imbalance is shown below

In [8]:
df['is_churned'].value_counts(normalize=True).mul(100).reset_index()

,is_churned,proportion
0,False,83.934038
1,True,16.065962


**Observaitons**

1. Credit card churn is observed in 16% of customers.

One common way to calculate the customer churn impact is to calculate the lost Customer Lifetime Value (CLV) per churned customer and add the replacement cost (CAC). The average CAC for a credit card customer is approximately [167](https://firstpagesage.com/seo-blog/average-customer-acquisition-cost-cac-in-banking/) USD or ~200 CAD. The total impact of churn is estimated below for sample of data provided by McMaster Bank

In [9]:
(
    df
    .groupby('is_churned')
    .agg({'clv': ['sum', 'count']})
    .set_axis(['clv_total', 'num_customers'], axis=1)
    .reset_index()
    .assign(
        clf_fraction=lambda df: (
            (df['clv_total']/df['clv_total'].sum()).mul(100)
        ),
        clv_per_customer=lambda df: df['clv_total']/df['num_customers'],
    )
    .query("is_churned == True")
    .reset_index(drop=True)
    .assign(
        impact_per_customer=lambda df: df['clv_per_customer']+replacement_cost,
        impact_total=lambda df: (
            df['clv_total']+df['num_customers'].mul(replacement_cost)
        ),
    )
)

,is_churned,clv_total,num_customers,clf_fraction,clv_per_customer,impact_per_customer,impact_total
0,True,826428.2668,1627,9.98216,507.946077,707.946077,1151828.2668


**Observations**

1. Approximately 10% of customer lifetime value is lost due to the 16% churn rate over the past 12 months in the random sample of customer data. The impact to the client (i.e. to the bank's credit card division) is a loss of approximately 508 dollars of customer lifetime value (per customer). When the cost to acquire a new customer is taken into account this adds up to a loss of 708 dollars per customer, or 1,151,828 dollars overall, due to churn.